In [1]:
import requests
import pandas as pd

def fetch_race_results(season, round):
    url = f"http://ergast.com/api/f1/{season}/{round}/results.json"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def parse_race_results(data):
    race_info = data['MRData']['RaceTable']['Races'][0]
    race_name = race_info['raceName']
    date = race_info['date']
    results = race_info['Results']
       
    parsed_results = []
    for result in results:
        parsed_results.append({
            'position': result['position'],
            'driver': f"{result['Driver']['givenName']} {result['Driver']['familyName']}",
            'constructor': result['Constructor']['name'],
            'grid': result['grid'],
            'laps': result['laps'],
            'status': result['status'],
            'time': result['Time']['time'] if 'Time' in result else None,
            'points': result['points']
        })
       
    return race_name, date, pd.DataFrame(parsed_results)

if __name__ == "__main__":
    season = 2023
    round = 1
    data = fetch_race_results(season, round)
    if data:
        race_name, date, df = parse_race_results(data)
        print(f"Race: {race_name} on {date}")
        print(df)
    else:
        print("Failed to fetch data")


Race: Bahrain Grand Prix on 2023-03-05
   position           driver     constructor grid laps      status  \
0         1   Max Verstappen        Red Bull    1   57    Finished   
1         2     Sergio Pérez        Red Bull    2   57    Finished   
2         3  Fernando Alonso    Aston Martin    5   57    Finished   
3         4     Carlos Sainz         Ferrari    4   57    Finished   
4         5   Lewis Hamilton        Mercedes    7   57    Finished   
5         6     Lance Stroll    Aston Martin    8   57    Finished   
6         7   George Russell        Mercedes    6   57    Finished   
7         8  Valtteri Bottas      Alfa Romeo   12   57    Finished   
8         9     Pierre Gasly  Alpine F1 Team   20   57    Finished   
9        10  Alexander Albon        Williams   15   57    Finished   
10       11     Yuki Tsunoda      AlphaTauri   14   57    Finished   
11       12   Logan Sargeant        Williams   16   56      +1 Lap   
12       13  Kevin Magnussen    Haas F1 Team   17  

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
%pip install fetch_race_results
from fetch_race_results import fetch_race_results, parse_race_results

season = 2023
round = 1
data = fetch_race_results(season, round)
if data:
    race_name, date, df = parse_race_results(data)
    print(f"Race: {race_name} on {date}")
    display(df)

    # Example analysis: Points distribution
    df['points'] = df['points'].astype(float)
    df.plot(kind='bar', x='driver', y='points', title='Points Distribution')
    plt.show()
else:
    print("Failed to fetch data")


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement fetch_race_results (from versions: none)
ERROR: No matching distribution found for fetch_race_results


ModuleNotFoundError: No module named 'fetch_race_results'

In [3]:
from flask import Flask, render_template
import requests
import pandas as pd

app = Flask(__name__)

def fetch_race_results(season, round):
    url = f"http://ergast.com/api/f1/{season}/{round}/results.json"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def parse_race_results(data):
    race_info = data['MRData']['RaceTable']['Races'][0]
    race_name = race_info['raceName']
    date = race_info['date']
    results = race_info['Results']
       
    parsed_results = []
    for result in results:
        parsed_results.append({
            'position': result['position'],
            'driver': f"{result['Driver']['givenName']} {result['Driver']['familyName']}",
            'constructor': result['Constructor']['name'],
            'grid': result['grid'],
            'laps': result['laps'],
            'status': result['status'],
            'time': result['Time']['time'] if 'Time' in result else None,
            'points': result['points']
        })
       
    return race_name, date, pd.DataFrame(parsed_results)

@app.route('/')
def index():
    season = 2023
    round = 1
    data = fetch_race_results(season, round)
    if data:
        race_name, date, df = parse_race_results(data)
        return render_template('index.html', race_name=race_name, date=date, tables=[df.to_html(classes='data')], titles=df.columns.values)
    else:
        return "Failed to fetch data"

if __name__ == "__main__":
    app.run(debug=True)


ModuleNotFoundError: No module named 'flask'